# Bem vindos ao primeiro encontro de fundamentos de python!

## Sobre o Jupyter Lab


Estamos trabalhando aqui em uma ferramenta chamada **Jupyter Lab**, a qual é uma das formas mais conhecidas de se executar códigos em python. Inclusive com o Jupyter, você não tem acesso só ao kernel do python. O próprio nome Jupyter vem da junção de algumas linguagens que ele possui suporte:

- JU (Julia)
- PY (Python)
- (te)R (R)

Por curiosidade, [aqui tem uma lista](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels) das linguagens de programação suportadas pelo Jupyter.

## Formas clássicas de se executar códigos python

Trabalhar com notebooks jupyter é ótimo para experimentar código, dado que é talvez a plataforma mais interativa de python. Entretanto relatarei aqui outras formas clássicas de se executar códigos python.

#### Para criar (e debugar) códigos python:

- IDE (como o [VSCode](https://code.visualstudio.com/)) - Possivelmente a minha forma favorita de interagir com código python, especialmente em virtude das [ricas funcionalidades de debugging](https://code.visualstudio.com/docs/python/debugging).
- [Python Interpreter](https://docs.python.org/3/tutorial/interpreter.html) - Via terminal é possível acessar um interpretador python, o qual possui um modo interativo, aonde é possível rapidamente experimentar código.
- Jupyter Lab - Plataforma que estaremos utilizando hoje, executa código python de forma interativa.

#### Para executar em produção:
- Executar o [script python via terminal](https://pythonbasics.org/execute-python-scripts/), com um comando `python nome_script.py`.
- Executar o script python via terminal dentro de um container. [Link](https://hub.docker.com/_/python) e [Link](https://www.docker.com/blog/how-to-dockerize-your-python-applications/)
- Criar uma webserver em python, tipo em [Flask](https://flask.palletsprojects.com/en/2.3.x/) ou [Django](https://www.djangoproject.com/).
- Confesso que criar [GUI's](https://realpython.com/python-gui-tkinter/) tem sido cada menos usado, mas existe essa possibilidade também em python.

## Algumas funcionalidades do Jupyter Lab


O Jupyter lab é dividido em células, as quais podem possuir tanto texto em markdown, como esse aqui, quanto código em uma linguagem de programação. Markdown é uma linguagem de marcação super utilizada no mundo tech para estilizar textos usando apenas texto. [Segue aqui um tutorial bacana de Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

## Revisando Python

Primeiramente, para uma revisão geral de Python, recomendo a realização do Tutorial de Python da W3 Schools, [a qual pode ser acessado aqui](https://www.w3schools.com/python/).

A maneira como eu gosto de enxergar Python é que **quase tudo em python é um objeto, os quais possuem propriedades e qualidades específicas**. Então, ao conhecer os principais objetos, suas características e propriedades, consequentemente você se tornará um melhor programador python.

Os tipos principais de dados em python, baseado na minha experiência, são:
- string (str): para armazenar textos
- integer (int): para armazenar números inteiros
- float (float): para armazenar números quebrados
- Decimal (Decimal): para armazenar números decimais só quando necessário, tipo quando lidar com arquivos parquet.
- datetime (datetime.datetime): para armazenar datas

Os tipos de dados de conjuntos de dados para armazenar dados em python:
- list (list) []: para armazenar uma sequência de dados
- dict (dict) {}: para armazenar um dicionário de dados, chave e valor.
- DataFrame (pandas.DataFrame): para armazenar em memória uma tabela.

In [19]:
# Nas células de código, o # indica que inicia-se um comentário.

In [ ]:
# Exemplo de request para pegar dados da WEB:

In [20]:
import requests

In [21]:
search = "feijão"
url = f"https://www.supermuffato.com.br/{search}"

In [39]:
result = requests.get(url)
result_str = result.text

In [23]:
# Salvando dados do HTML em disco, uma vez que essa informação
# é bastante volátil, e qualquer tratamento nesse dado poderia
# quebrar o pipeline. Salvando em disco o dado bruto permite
# a gente reprocessar o dado posteriormente, se necessário.

In [36]:
from pathlib import Path
from datetime import datetime
import uuid

In [37]:
date = datetime.now().strftime('%Y-%m-%d')
id = uuid.uuid4()

directory_path = Path(f'downloads/brass/muffato_html/search={search}/date={date}/')

# Por padrão não se cria uma cadeia de diretórios ao criar um arquivo,
# o comando abaixo faz isso

directory_path.mkdir(parents=True, exist_ok=True)

path = directory_path / f'{id}.html'

In [40]:
with open(path, 'w+') as f:
    f.write(result.text)

In [41]:
# Parseando dados em HTML

In [46]:
from bs4 import BeautifulSoup, Tag

In [45]:
def get_nested_dict_attr_value(tag: Tag, attr: str):
    attr_dict = {}
    if hasattr(tag, 'children'):
        for subtag in tag.children:
            if hasattr(subtag, 'attrs') and attr in subtag.attrs:
                classes = " ".join(subtag.attrs[attr])
                attr_dict[classes] = subtag.get_text()
            attr_dict.update(get_nested_dict_attr_value(subtag, attr))
    return attr_dict

In [47]:
soup = BeautifulSoup(result_str)

In [51]:
# Primeiro passo é filtrar para pegar os elementos que
# são de nosso interesse.

products = soup.findAll("div", {"class": "prd-list-item-holder"})

In [56]:

# Perceba aqui as estruturas de dados. As listas
# seriam como as linhas de uma tabela
products_data = []
for product in products:
    # Enquanto que as colunas seriam os dicionários
    # como o dicionário gerado pela função get_nested_dict_attr_value
    products_data.append(get_nested_dict_attr_value(product, 'class'))

# Essa estrutura de lista de python como linha e dicionário
# de python como coluna é um padrão muito comumente utilizado
# em programas de python para dados.

In [58]:
# Acredito que JSON seja o tipo de dado mais usado e mais útil
# no universo WEB. Perceba que Python possui suporte nativo
# ao tipo de dado JSON por meio da biblioteca JSON

# Com ela é possível ler com facilidade dados em JSON (convertendo
# em dicionário Python) e o caminho contrario também (de dicionário
# python para JSON).

# Inclusive uma lista de dicionários é possível se converter em JSON
# é o que faremos agora.

import json

directory_path = Path(f'downloads/bronze/muffato_html/search={search}/date={date}/')
directory_path.mkdir(parents=True, exist_ok=True)
path = directory_path / f'{id}.json'
with open(path, 'w+') as f:
    json.dump(products_data, f)